In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!mkdir scene_data

!cp drive/MyDrive/Scene.zip /content/scene_data

Mounted at /content/drive/


In [ ]:
cd scene_data/

/content/scene_data


In [ ]:
!unzip Scene.zip

!rm -r Scene.zip

Archive:  Scene.zip
   creating: 15-Scene/15-Scene/
   creating: 15-Scene/15-Scene/00/
  inflating: 15-Scene/15-Scene/00/1.jpg  
  inflating: 15-Scene/15-Scene/00/10.jpg  
 extracting: 15-Scene/15-Scene/00/100.jpg  
 extracting: 15-Scene/15-Scene/00/101.jpg  
 extracting: 15-Scene/15-Scene/00/102.jpg  
 extracting: 15-Scene/15-Scene/00/103.jpg  
 extracting: 15-Scene/15-Scene/00/104.jpg  
 extracting: 15-Scene/15-Scene/00/105.jpg  
 extracting: 15-Scene/15-Scene/00/106.jpg  
 extracting: 15-Scene/15-Scene/00/107.jpg  
 extracting: 15-Scene/15-Scene/00/108.jpg  
 extracting: 15-Scene/15-Scene/00/109.jpg  
  inflating: 15-Scene/15-Scene/00/11.jpg  
 extracting: 15-Scene/15-Scene/00/110.jpg  
 extracting: 15-Scene/15-Scene/00/111.jpg  
 extracting: 15-Scene/15-Scene/00/112.jpg  
 extracting: 15-Scene/15-Scene/00/113.jpg  
 extracting: 15-Scene/15-Scene/00/114.jpg  
 extracting: 15-Scene/15-Scene/00/115.jpg  
 extracting: 15-Scene/15-Scene/00/116.jpg  
  inflating: 15-Scene/15-Scene/00/117

In [ ]:
import torch
import os
import pandas as pd
import numpy as np
import warnings
# from functions import train_model, evaluate_model, detailed_evaluation, read_config
from torchvision import models
from torchvision.models.efficientnet import EfficientNet_B0_Weights
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


class CustomDataset(Dataset):
    def __init__(self, img_dir, transform):
        super().__init__()
        self.img_dir = img_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for label in os.listdir(self.img_dir):
            img_folder_path = os.path.join(self.img_dir, label)
            for img in os.listdir(img_folder_path):
                img_path = os.path.join(img_folder_path, img)
                self.images.append(img_path)
                self.labels.append(int(label))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


def train_model(model, dataloader, validation_loader, epochs, optimizer, entropyloss):
    print("\n-----------   Model training   -----------\n")
    best_accuracy = 0
    best_model = None
    size_loader = len(dataloader)
    for epoch in range(epochs):
        running_loss = 0.0
        model.train()  # set model to training mode
        for images, labels in dataloader:
            prediction = model(images) # predicted outputs
            loss = entropyloss(prediction, labels) # entropy loss

            optimizer.zero_grad() # reset the gradients of model parameters
            loss.backward() # backpropagate the prediction loss
            optimizer.step() # adjust the parameters by the gradients collected in the backward pass

            running_loss += loss.item()

        epoch_loss = running_loss / size_loader
        validation_loss, validation_accuracy = validate_model(model, validation_loader, entropyloss)
        print(f'Epoch {epoch + 1}, Train Loss: {epoch_loss:.4f}, Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}')

        # Save the model if it has the best accuracy so far
        if validation_accuracy > best_accuracy:
            best_accuracy = validation_accuracy
            best_model = model.state_dict()  # save the model parameters
            print("Best model updated")

    print("Training complete. Best Accuracy: {:.2f}%".format(best_accuracy))
    return best_model


def validate_model(model, validation_loader, entropyloss):
    print("\n-----------   Model validation   -----------\n")
    model.eval()  # Set model to evaluation mode
    validation_loss = 0.0
    correct = 0
    total = 0
    size_loader = len(validation_loader)

    with torch.no_grad():
        for images, labels in validation_loader:
            prediction = model(images)
            validation_loss += entropyloss(prediction, labels).item()

            correct += (prediction.argmax(1) == labels).type(torch.float).sum().item()
            total += labels.size(0)

    validation_loss /= size_loader
    validation_accuracy = correct / total
    return validation_loss, validation_accuracy


def evaluate_test(model, dataloader, entropyloss):
    print("\n-----------   Test evaluation   -----------\n")
    model.eval()
    all_predictions = []
    all_labels = []
    total_loss = 0.0
    size_loader = len(dataloader)

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            loss = entropyloss(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / size_loader
    accuracy = accuracy_score(all_labels, all_predictions)
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(classification_report(all_labels, all_predictions))
    print("Confusion Matrix:\n", confusion_matrix(all_labels, all_predictions))

In [ ]:
warnings.filterwarnings('ignore')
epochs=10
learning_rate = 1e-3
batch_size = 64
data_path = '15-Scene/15-Scene'
# config = read_config('config.txt')
# epochs = config['epochs']
# learning_rate = config['learning_rate']
# batch_size = config['batch_size']
# data_path = config['data_path']

# def read_config(file_path): # read configuration file
#     config = {}
#     with open(file_path, 'r') as file:
#         for line in file:
#             key, value = line.strip().split('=')
#             config[key.strip()] = value.strip()
#     return config


# load pre-trained EfficientNet-B0
efficientnet = models.efficientnet_b0(weights=True, pretrained=True)

# make all layers trainable
for param in efficientnet.parameters():
    param.requires_grad = True

transform = EfficientNet_B0_Weights.IMAGENET1K_V1.transforms()
# optimizer and entropy loss function used
optimizer = torch.optim.Adam(efficientnet.parameters(), lr=learning_rate)
entropyloss = torch.nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 86.9MB/s]


In [ ]:
# ########### FULLY SUPERVISED MODEL

scene_data = CustomDataset(img_dir=data_path, transform=transform) # load data from the folders into scene_data

# Obtain labels for each index
total_size = len(scene_data)
indices = list(range(total_size))
labels = [scene_data[i][1] for i in indices]

# Stratified split using scikit-learn's train_test_split
train_set_idx, validation_set_idx, _, _ = train_test_split(indices, labels, test_size=0.3, random_state=42, stratify=labels)
validation_set_idx, test_set_idx, _, _ = train_test_split(validation_set_idx, [labels[i] for i in validation_set_idx], test_size=0.5, random_state=42, stratify=[labels[i] for i in validation_set_idx])

# Create subset data loaders
train_dataset = Subset(scene_data, train_set_idx)
validation_dataset = Subset(scene_data, validation_set_idx)
test_dataset = Subset(scene_data, test_set_idx)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# ######### FULLY SUPERVISED

# # train the model with the train loader in how many epochs you want
# best_model = train_model(model=efficientnet, dataloader=train_loader, validation_loader=validation_loader, epochs=epochs, optimizer=optimizer, entropyloss=entropyloss)
# efficientnet.load_state_dict(best_model)  # Load the best model parameters

# # Call the detailed evaluation function
# evaluate_test(efficientnet, test_loader)


In [ ]:
from PIL import Image
import cv2
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms, models
from torchvision.models.efficientnet import EfficientNet_B0_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CustomBlurDataset(Dataset):
    def __init__(self, img_dir, transform):
        super().__init__()
        self.img_dir = img_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.kernel_sizes = [(5, 5), (9, 9), (13, 13), (17, 17), (21, 21)]

        for label in os.listdir(self.img_dir):
            img_folder_path = os.path.join(self.img_dir, label)
            for img in os.listdir(img_folder_path):
                img_path = os.path.join(img_folder_path, img)
                self.images.append(img_path)
                self.labels.append(int(label))

    def __len__(self):
        return len(self.images) * len(self.kernel_sizes) #

    def __getitem__(self, idx):
        img_idx = idx // len(self.kernel_sizes)
        kernel_idx = idx % len(self.kernel_sizes)

        image = Image.open(self.images[img_idx]).convert('RGB')
        kernel_size = self.kernel_sizes[kernel_idx]

        image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        blurred_image = cv2.GaussianBlur(image_np, kernel_size, sigmaX=0)
        blurred_image = Image.fromarray(cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB))

        if self.transform:
            blurred_image = self.transform(blurred_image)

        label = kernel_idx
        return blurred_image, label


def train_blurred_model(model, dataloader, epochs, optimizer, entropyloss):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            prediction = model(images) # predicted outputs
            loss = entropyloss(prediction, labels) # entropy loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(prediction, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(dataloader)
        epoch_accuracy = correct / total
        print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

        return model



In [ ]:
############# BLUR
import torch.nn as nn

# # load pre-trained EfficientNet-B0
# efficientnet = models.efficientnet_b0(weights=True, pretrained=True)

# # make all layers trainable
# for param in efficientnet.parameters():
#     param.requires_grad = True

# transform = EfficientNet_B0_Weights.IMAGENET1K_V1.transforms()
# # optimizer and entropy loss function used
# optimizer = torch.optim.Adam(efficientnet.parameters(), lr=learning_rate)
# entropyloss = torch.nn.CrossEntropyLoss()


# Modify the classifier to predict the blur kernel size
num_features = efficientnet.classifier[1].in_features
efficientnet.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(num_features, 5)  # 5 classes for the 5 kernel sizes
)
efficientnet.to(device)

# Create the dataset
blurred_data = CustomBlurDataset(img_dir=data_path, transform=transform)

# Create data loaders
train_blurred_loader = DataLoader(blurred_data, batch_size=batch_size, shuffle=True)

efficientnet = train_blurred_model(model=efficientnet, dataloader=train_blurred_loader, epochs=epochs, optimizer=optimizer, entropyloss=entropyloss)

num_scene_classes = 15           ################# SOSSSS FIX IT DYNAMICALLY ###################
efficientnet.classifier = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(num_features, num_scene_classes)
)

efficientnet = train_blurred_model(model=efficientnet, dataloader=train_loader, epochs=epochs, optimizer=optimizer, entropyloss=entropyloss)

evaluate_test(efficientnet, test_loader)



# # pick a picture
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]

img_np = img.permute(1, 2, 0).cpu().numpy()  # Reorder the dimensions
img_np = (img_np * 0.5) + 0.5
print(f"Label: {label}")
plt.imshow(img_np, cmap="gray")
plt.show()








